In [1]:
import os
import fitz  # PyMuPDF for reading PDFs
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import gradio as gr

# 🔑 Set OpenAI API key
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

# 🧠 Setup LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 📄 Prompt Template
template = """You are a professional resume analyst. Analyze the following resume text and provide:
- A summary of candidate's profile
- Key skills identified
- Experience level
- Education background
- Suggestions for improvement (if any)

Resume:
{text}
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template
)

# 🔗 LangChain LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# 📥 Read PDF content
def extract_text_from_pdf(pdf_file):
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# 🎯 Analysis function
def analyze_resume(file):
    if file is None:
        return "Please upload a resume."

    # Extract text from uploaded PDF
    text = extract_text_from_pdf(file)
    if len(text.strip()) == 0:
        return "Could not extract text from the uploaded file."

    # Analyze with LLM
    result = chain.run(text=text)
    return result

# 🎨 Gradio UI
gr.Interface(
    fn=analyze_resume,
    inputs=gr.File(label="Upload Resume (PDF)", type="binary"),
    outputs="text",
    title="📄 LangChain Resume Analyzer",
    description="Upload a resume in PDF format. The AI will analyze skills, experience, education, and suggest improvements."
).launch()


ModuleNotFoundError: No module named 'fitz'